<a href="https://colab.research.google.com/github/Jonathan-C-Barrett/GEOG5990M/blob/main/GEOG5003M_Final_Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Final Assignment

Student ID number: 201804755

GitHub repo link: https://github.com/Jonathan-C-Barrett/GEOG5990M/tree/main

Word count limit= 1,500 words max (markdown cells only, excluding readme)

# Read in Packages

In [ ]:
#Install required packages
!pip install mapclassify
!pip install contextily
!pip install geoplot
!pip install git+https://github.com/pmdscully/geo_northarrow.git


# read in required packages
import geopandas as gpd
import pandas as pd
# remove default='warn' for chained assigments
pd.options.mode.chained_assignment = None
import seaborn as sns
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pyproj
import contextily as ctx
import geoplot as gplt
import geoplot.crs as gcrs
from geo_northarrow import add_north_arrow



# import the required machine learning packages
from sklearn import cluster
from sklearn.preprocessing import scale





# Pre-processing

The following was required for pre-processing.

First was the selection of appropriate variables from the access to services dataset. These were selected based on services directly linked to personal health, including leisure facilities which support healthiness <a href="https://www.nature.com/articles/s41597-019-0114-6">[1]</a>. Using these five variables a weighted average was created, with weights based on previous studies of services with the largest impact <a href="https://www.nature.com/articles/s41597-019-0114-6">[1]</a>, and stored under the code ‘ah4h’.

Second was to deal with missing values, either from individual quarters or the entire year, of the median property data. A yearly mean column under ‘2016_mean’ was created and populated with the mean value of the combined quarterly values. This negated the issue with missing quarterly data while additionally benefiting from the removal of seasonal impact on house pricing <a href="http://dx.doi.org/10.2139/ssrn.2785400">[2]</a>. After creating the mean value, for any rows that remained missing due to a complete missing year, these were removed as there was no way to extrapolate the data accurately from elsewhere.

Following this area codes were converted from LSOA11 to LSOA21 by combining the data with a conversion data frame, where we then also removed unnecessary columns to make the data frame less cluttered.

To investigate outliers in the data, descriptive statistics, pair plots and box plots were used. Although all data sets showed some evidence of extreme values, property value showed extreme outliers at the larger end and so these were identified and removed using the interquartile range method. This method was used due to its simplicity and reproducibility at removing outliers so as not to distort the results of the correlation <a href="https://www.sciencedirect.com/science/article/pii/S2772662223000048">[3]</a>.


**HEALTH SERVICE ACCESS DATA**

In [ ]:
#Read in Health Services Data
#Data sourced from: https://data.geods.ac.uk/dataset/access-to-healthy-assets-hazards-ahah
ahah = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/ahah_v4.csv')
ahah.head()

In [ ]:
#Select only columns relating to Health Services
health = ahah[['LSOA21CD', 'ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis']]
health.head()

In [ ]:
#check for missing values
health.isna().sum()

In [ ]:
#explore general statistics
health.describe()

In [ ]:
#create weighted average score column of services
weights = [0.2,0.2,0.2,0.15,0.15]
health.loc[:,'ah4h'] = (health[['ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis']] * weights).sum(axis=1)
health

**MEDIAN PROPERTY VALUE DATA**

In [ ]:
#Read in Median Property Price
#data sourced from: https://data.london.gov.uk/dataset/average-house-prices
m_value = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/land-registry-house-prices-Meidan-LSOA.csv')
m_value.head()

In [ ]:
#Select only columns relating to 2016
prop_16 = m_value[['Code', 'Area','Year ending Mar 2016','Year ending Jun 2016', 'Year ending Sep 2016', 'Year ending Dec 2016']]
prop_16.head()

In [ ]:
#examine property info for count and data type
prop_16.info()

In [ ]:
#check for missing values
prop_16.isna().sum()

In [ ]:
#create a column of mean year value to counter missing values
prop_16.loc[:,'2016_mean'] = prop_16[['Year ending Mar 2016', 'Year ending Jun 2016', 'Year ending Sep 2016', 'Year ending Dec 2016']].mean(axis=1)
prop_16.head()

In [ ]:
prop_16.isna().sum()

**LSOA 2011 TO LSOA 2021 COVERSION DATA**

In [ ]:
#add conversion to LSOA21
#Data sourced from: https://geoportal.statistics.gov.uk/datasets/b14d449ba10a48508bd05cd4a9775e2b_0/explore
LSOA21 = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Best_Fit_Lookup_for_EW_(V2).csv')
LSOA21.head()


In [ ]:
#inspect info for data type
LSOA21.info()

In [ ]:
#combine prop_16 data and LSOA21 by code in prop_16 and LSOA11CD in LSOA21 in to remove areas outside london and make comparable
prop_16_LSOA21 = pd.merge(prop_16, LSOA21, left_on='Code', right_on='LSOA11CD', how='left')
prop_16_LSOA21.head()

In [ ]:
#check Info for count
prop_16_LSOA21.info()

**COMBINE HEALTH ACCESS AND PROPERTY VALUE DATABASES**

In [ ]:
#combine health and property data by LSOA code in property to remove areas outside london
health_prop = pd.merge(health, prop_16_LSOA21, left_on='LSOA21CD', right_on='LSOA21CD', how='right')
health_prop.head()

In [ ]:
#check info for data frame to check count and data type
health_prop.info()

In [ ]:
#check for missing values
health_prop.isnull().sum()

In [ ]:
#remove missing values from Year mean column
health_prop = health_prop.dropna(subset=['2016_mean'])

In [ ]:
#Check removed 2016 Mean missing values
health_prop.isnull().sum()

In [ ]:
#remove duplicated and unneccessary columns from Data base
health_prop_clean = health_prop.drop(columns=['Code', 'Area', 'LSOA11NM', 'LSOA11CD', 'ObjectId','LAD22NMW', 'Year ending Mar 2016', 'Year ending Jun 2016','Year ending Sep 2016','Year ending Dec 2016', ])
health_prop_clean.head()


In [ ]:
#check data type and missing values of required columns
health_prop_clean.info()


**INVESTIGATE DISTRIBUTION OF DATA AND REMOVAL OF OUTLIERS**

In [ ]:
#check basic geometry of data
health_prop_clean.describe()

In [ ]:
#Explore correlation between property price mean and health service float data with pairplot
sns.pairplot(health_prop_clean[['ah4dent','ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h','2016_mean']]);

In [ ]:
#check for outliers observed in pairplot
health_prop_clean['2016_mean'].nlargest(n=25)

In [ ]:
#use boxplot to inspect impact of outliers
sns.boxplot(data=health_prop_clean, x='2016_mean');
plt.gcf().axes[0].xaxis.get_major_formatter().set_scientific(False)


In [ ]:
sns.boxplot(data=health_prop_clean, x='ah4gp');

In [ ]:
sns.boxplot(data=health_prop_clean, x='ah4dent');

In [ ]:
sns.boxplot(data=health_prop_clean, x='ah4hosp');

In [ ]:
sns.boxplot(data=health_prop_clean, x='ah4leis');

In [ ]:
sns.boxplot(data=health_prop_clean, x='ah4h');

In [ ]:
#remove outliers in 2016 mean using interquartle range method
#find intterquartile range
Q1 = health_prop_clean['2016_mean'].quantile(0.25)
Q3 = health_prop_clean['2016_mean'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
#view Q1, Q3 and IQR
Q1, Q3, IQR

In [ ]:
#find upper and lower limits
upper_limit = Q3 + (1.5 * IQR)
lower_limit = Q1 - (1.5 * IQR)
lower_limit, upper_limit

In [ ]:
#find outliers
outliers = health_prop_clean[(health_prop_clean['2016_mean'] > upper_limit) | (health_prop_clean['2016_mean'] < lower_limit)]
outliers

In [ ]:
#trim the data
health_prop_clean_trim = health_prop_clean[(health_prop_clean['2016_mean'] < upper_limit) & (health_prop_clean['2016_mean'] > lower_limit)]


In [ ]:
#check for difference in data
print('before removing outliers', health_prop_clean.shape)
print('after removing outliers', health_prop_clean_trim.shape)

# Data Exploration

The correlation between variables were explored at an LSOA scale using spearman’s rank. This was chosen as a simple statistical measure of correlation that does not require assumptions about distribution <a href="https://onlinelibrary.wiley.com/doi/10.1002/cpe.3745">[4]</a> The result of this were displayed on a heatmap with a mask covering duplicate records to make the data easier to interpret.  A color scheme of ‘RdBlu’ enabled easy identification of a positive or negative correlation. The rank value was also added to each square for clarity to the audience.  



**CORRELATION THROUGH SPEARMANS RANK**

In [ ]:
#Calculate Spearmans Rank Correlation for all float data

health_prop_corr =health_prop_clean_trim[['ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h','2016_mean']].corr(method = 'spearman')
health_prop_corr


In [ ]:
#Visualize spearmans correlation

# define plot size
fig,ax = plt.subplots(figsize=(8,8))

# define mask to apply to upper right hand corner of the plot
data_to_mask = np.triu(np.ones_like(health_prop_corr))

# define axis labels
x_axis_labels = health_prop_corr.columns
y_axis_labels = health_prop_corr.index


#capatlise each label is x and Y
x_axis_labels = [element.capitalize() for element in x_axis_labels]
y_axis_labels = [element.capitalize() for element in y_axis_labels]

# plot a heatmap of the correlation
sns.heatmap(health_prop_corr,
            #exband linewith for clarity
            linewidths=.5,
            # include spearmans value in squares
            annot=True,
            # define colourmap
            cmap='RdBu',
            # define value of minimum colour on cbar
            vmin=-1,
            # define value of maximum colour on cbar
            vmax=1,
            # add the mask
            mask=data_to_mask,
            # use the custom labels
            xticklabels=x_axis_labels,
            yticklabels=y_axis_labels,
            # add a label to the cbar
            cbar_kws={'label': "Spearman's Rank correlation"},
            # plot on the axis we defined
            ax=ax)

# Set axis labels
ax.set(xlabel="Property Value and Access to Health Services",
       ylabel="Property Value and Access to Health Services",
      title ='Property Value and Access to Health Services Correlation' );


**Create Inner and Outer boruough classification**

In [ ]:
#create column of inner or outer borough for additional information in visualisation
inner_london_boroughs = ('Camden', 'City of London', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Islington', 'Kensington and Chelsea', 'Lambeth', 'Lewisham', 'Southwark', 'Tower Hamlets', 'Wandsworth', 'Westminster')

for index, row in health_prop_clean_trim.iterrows():
  if row['LAD22NM'] in inner_london_boroughs:
    health_prop_clean_trim.loc[index, 'inner_outer'] = 'Inner Borough'
  else:
    health_prop_clean_trim.loc[index, 'inner_outer'] = 'Outer Borough'

In [ ]:
#check inner_outer has worked correctly
num_unique_categories = health_prop_clean_trim['inner_outer'].nunique()
print(f"Number of unique categories: {num_unique_categories}")

# Non Spatial Visulation

The first visualization displays the relationship between median property value and individual variables of health access. Five smaller scatterplots have been included for each health service and one larger scatter plot for the overall weighted average access.

A scatterplot allows the relationship of two numerical variables to be clearly identified in a style easily recognizable to a non-technical audience <a href="https://ijcttjournal.org/archives/ijctt-v71i4p116">[5]</a>, and for each plot a regression line was added to easily identify the direction of the correlation. For the larger plot histograms of both variables were included on the secondary x and y axes to provide clarity on the individual distribution of each.

For additional detail the inner and outer boroughs of London were identified and added to the hue of the visualization. This adds additional information for the audience on the distribution of these points across London.

The key audience of this visualization is non-technical regional or national decision makers, who would assess the need for additional infrastructure in London while possibly lacking the expertise to deal with large data <a href="https://ijcttjournal.org/archives/ijctt-v71i4p116">[5]</a>. As such, when creating this visualization, accessibility and ease of interpretation were key drivers in many of the choices.

The color scheme of ‘colorblind’ was chosen to remain accessible to all, and hexagonal point markers used as often circular markers can be difficult to identify when overlapping <a href="https://data.europa.eu/apps/data-visualisation-guide/handling-overlap-in-scatter-plots">[6]</a>. In addition, we also added a level of opacity that ensures the points are less cluttered. We also reduced the amount of tick markers and included only one x and y axis title on the smaller graphs for a clearer aesthetic, and finally changed the orientation of the tick labels on the larger plot to ensure the visualization remained uncluttered and easy to interpret.


In [ ]:
#create four subplots of health service individual variables
f,ax = plt.subplots(2,3, figsize=(12,6))
#widen space between subplots
plt.subplots_adjust(wspace=0.4, hspace=0.4)

#Add title to subplots
f.suptitle('Mean Property Value Vs Health Access Variables ', fontsize=16)


#plot distance to gp in plot 1
sns.scatterplot(data=health_prop_clean_trim, x='2016_mean', y='ah4gp', hue='inner_outer', legend=False, marker= "h", palette='colorblind', alpha =0.7, ax=ax[0,0], ).locator_params(axis='x', nbins=3)
#Add Title to plot
ax[0, 0].set_title('Access to GP')
#add regression line
sns.regplot(data=health_prop_clean_trim, x='2016_mean', y="ah4gp", scatter=False, ax=ax[0,0], color='red')
#remove xaxis title
ax[0,0].set_xlabel('')
#remove yaxis title
ax[0,0].set_ylabel('')



#plot distance to hospital in plot 2
sns.scatterplot(data=health_prop_clean_trim, x='2016_mean', y='ah4hosp', hue='inner_outer', legend=False, marker= "h", palette='colorblind', alpha =0.7, ax=ax[0,1]).locator_params(axis='x', nbins=3)
#Add title to plot
ax[0, 1].set_title('Access to Hospital')
#add regression line
sns.regplot(data=health_prop_clean_trim, x='2016_mean', y="ah4hosp", scatter=False, ax=ax[0,1], color='red')
#remove xaxis title
ax[0,1].set_xlabel('')
#remove yaxis title
ax[0,1].set_ylabel('')

#plot distance to hospital in plot 3
sns.scatterplot(data=health_prop_clean_trim, x='2016_mean', y='ah4dent', hue='inner_outer', legend=False, marker= "h", palette='colorblind', alpha =0.7, ax=ax[0,2]).locator_params(axis='x', nbins=3)
#Add title to plot
ax[0,2].set_title('Access to Dentist')
#add regression line
sns.regplot(data=health_prop_clean_trim, x='2016_mean', y="ah4dent", scatter=False, ax=ax[0,2], color='red')
#remove xaxis title
ax[0,2].set_xlabel('')
#remove yaxis title
ax[0,2].set_ylabel('')



#plot distance to pharmacy in plot 4
sns.scatterplot(data=health_prop_clean_trim, x='2016_mean', y='ah4phar', hue='inner_outer', legend=False,  marker= "h", palette='colorblind', alpha =0.7, ax=ax[1,0]).locator_params(axis='x', nbins=3)
#add plot title
ax[1,0 ].set_title('Access to Pharmacy')
#add regression line
sns.regplot(data=health_prop_clean_trim, x='2016_mean', y="ah4phar", scatter=False, ax=ax[1,0], color='red')
#remove xaxis title
ax[1,0].set_xlabel('')
#center y label to use for four plots
ax[1,0].set_ylabel('Drive Time to Service (minutes)', y=1.25,fontsize=10)

#plot distance to leisure center in plot 5
sns.scatterplot(data=health_prop_clean_trim, x='2016_mean', y='ah4leis', hue='inner_outer', legend=False,  marker= "h", palette='colorblind', alpha =0.7, ax=ax[1,1]).locator_params(axis='x', nbins=3)
#add plot title
ax[1, 1].set_title('Access to Leisure Center')
#add regression line
sns.regplot(data=health_prop_clean_trim, x='2016_mean', y="ah4leis", scatter=False, ax=ax[1,1], color='red')
#center x label to use for four plots
ax[1,1].set_xlabel('Median Property Value (GBP)', labelpad=10, x=.8,fontsize=10)
#remove yaxis title
ax[1,1].set_ylabel('')

#Hide axis of plot 6
ax[1,2].axis('off')



#LARGE PLOT


#plot overal health access score vs property value
g=sns.JointGrid(data=health_prop_clean_trim, x='2016_mean', y='ah4h', hue="inner_outer", height =8)

#add regression line
sns.regplot(data=health_prop_clean_trim, x='2016_mean', y="ah4h", scatter=False, ax=g.ax_joint, color='red')

#remove scientific formating to include full house price
plt.gcf().axes[0].xaxis.get_major_formatter().set_scientific(False)

#plot health access score vs property value as scatter plot
sns.scatterplot(data=health_prop_clean_trim, x='2016_mean', y='ah4h', hue='inner_outer', marker= "h", palette='colorblind', alpha =0.7, ax=g.ax_joint)

#plothistograms on x-axis showing distribution of house price
sns.histplot(data=health_prop_clean_trim, x='2016_mean', hue="inner_outer", palette='colorblind', ax=g.ax_marg_x)

#plot histogram on y-axis showing distribution of weighted health access
sns.histplot(data=health_prop_clean_trim, y='ah4h', hue="inner_outer", palette='colorblind', ax=g.ax_marg_y)

#Remove legend from axis histograms
g.ax_marg_y.legend_.remove()
g.ax_marg_x.legend_.remove()

#Remove title from main legend
g.ax_joint.get_legend().set_title("")

#add title to property value distribution and set size
g.ax_marg_x.set_title('Property Value Distribution', fontsize=10)

#add title to weighted average Health Access Score distribution, set size, rotate and align to axis using x and y
g.ax_marg_y.set_title('Weighted Average Access Distribution', fontsize=10, rotation=270, y=0.3, x=1.1)

#Edit Axis Labels and set size
g.ax_joint.set_ylabel('Weighted Average Health Access (minutes)', fontsize=10)
g.ax_joint.set_xlabel('2016 Median Property Value (GBP)', fontsize=10);

#change orientation of x-axis ticks
g.ax_joint.tick_params(axis='x', rotation=45)

#add title to map and set position and fontsize
g.ax_joint.set_title('2016 Median Property Value Vs Weight Average Health Access', fontsize=16, y=1.3);

# LOCAL AUTHORITY DISTRICT (LAD) VISUALISTAION

#Additional Pre-processing for Spatial Visualization

To investigate this correlation at a different spatial scale, some extra pre-processing was required. The first step was to group the data into Local Authority Districts (LADs). This was done through the LAD21 code and the variables aggregated by mean. Mean was chosen as the most appropriate due to the nature and distribution of the data.
Additionally, median property price was divided by 100,000 to bring it to a comparable value with other data sets to aid in visualization, and the CSV file was joined with the geojson file to enable an accurate spatial projection.


**COMBINING LAD BOUNDARY GEOMETRY AND HEALTH ACCESS Vs PROPERTY VALUE**

In [ ]:
#read in shape file of boundaries
# Data downloaded from https://geoportal.statistics.gov.uk/datasets/ons::local-authority-districts-may-2024-boundaries-uk-bfe-2/explore?location=51.468470%2C-0.040938%2C9.90
shp = gpd.read_file('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/main/Local_Authority_Districts_May_2024_Boundaries_UK_BFE_2410925873296837173.geojson')



In [ ]:
#inspect data
shp.head()

In [ ]:
#group property and health data by Local Authority District
health_prop_LAD = health_prop_clean_trim.groupby('LAD22NM').mean(numeric_only=True)
health_prop_LAD.head(n=35)

In [ ]:
#convert 2016 property mean to more easily comparable by dividing by /100,000
health_prop_LAD['2016_mean'] = health_prop_LAD['2016_mean']/100000
health_prop_LAD.head()


In [ ]:
#explore basic statistics
health_prop_LAD.describe()

In [ ]:
#Cobine property and Health access data with geoson LAD boundaries and check all 33 Boroughs present
health_prop_LAD_ldn = pd.merge(shp, health_prop_LAD, left_on='LAD24NM', right_on='LAD22NM', how='right')
health_prop_LAD_ldn.head(n=35)

In [ ]:
#check data on leaflet map
health_prop_LAD_ldn.explore()

#Exploration
To explore the effect of the spatial aggregation a spearman’s rank correlation was again undertaken using the same visual decisions on the heatmap to ensure accessibility of the results.

Following this a K means clustering was undertaken to group LADs together. The ‘elbow method’ was used to determine the optimal number of clusters where minimal variation was observed at the lowest cluster count. K-means was selected due to its wide use as an unsupervised learning algorithm with the elbow method being suitable for smaller k values <a href="http://166.62.7.99/assets/default/article/2020/10/22/article_1603378206.pdf">[7]</a>.   

Four clear clusters were identified, of which the median was then calculated and plotted on a bar chart to easily identify the characteristics of each cluster. An appropriate name was then assigned, to make the characteristics of each clear for the audience.


**EXPLORE CORRELATION AT LAD SCALE**

In [ ]:
#Examine correlation between data at LAD level using Spearmans rank
health_prop_LADcorr = health_prop_LAD_ldn[['ah4dent','ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h','2016_mean']].corr(method = 'spearman')
health_prop_LADcorr

In [ ]:
#visualise correlation
# define plot size
fig,ax = plt.subplots(figsize=(8,8))

# define mask to apply to upper right hand corner of the plot
data_to_mask = np.triu(np.ones_like(health_prop_LADcorr))

# define axis labels
x_axis_labels = health_prop_LADcorr.columns
y_axis_labels = health_prop_LADcorr.index


#capatlise each label is x and Y
x_axis_labels = [element.capitalize() for element in x_axis_labels]
y_axis_labels = [element.capitalize() for element in y_axis_labels]

# plot a heatmap of the correlation
sns.heatmap(health_prop_LADcorr,
            #exband linewith for clarity
            linewidths=.5,
            # include spearmans value in sqaures
            annot=True,
            # define colourmap
            cmap='RdBu',
            # define value of minimum colour on cbar
            vmin=-1,
            # define value of maximum colour on cbar
            vmax=1,
            # add the mask
            mask=data_to_mask,
            # use the custom labels
            xticklabels=x_axis_labels,
            yticklabels=y_axis_labels,
            # add a label to the cbar
            cbar_kws={'label': "Spearman's Rank correlation"},
            # plot on the axis we defined
            ax=ax)

# Set axis labels
ax.set(xlabel="Property Value and Access to Health Services",
       ylabel="Property Value and Access to Health Services",
      title ='Property Value and Access to Health Services Correlation' );


**K- Means Classification**

In [ ]:
#Decide on number of clusters through elbow method
# create an empty list
Sum_of_squared_distances = []

# get a range of numbers from 1 to 15
K = range(1,15)
#for each number in the range 1 to 15create a k-means model with that number of clusters and set a random state
for k in K:
    km = cluster.KMeans(n_clusters=k, init="random", random_state=123)
    # fit the model using the variables from database
    km = km.fit(health_prop_LAD_ldn[['ah4dent','ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis','2016_mean']].values)
    # calculate the sum of the squared distances and add this to the 'Sum_of_squared_distances' list
    Sum_of_squared_distances.append(km.inertia_)

# plot the sum of squared distances against the number of clusters
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# run the model with minimum needed for minimal change in this instance 4
km4 = cluster.KMeans(n_clusters=4,init="random", random_state=123)
km4cls = km4.fit(health_prop_LAD_ldn[['ah4dent','ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis','2016_mean']].values)

In [ ]:
#create a new column in db with cluster label
health_prop_LAD_ldn['cluster'] = km4cls.labels_
health_prop_LAD_ldn['cluster'].head()

In [ ]:
# Create a pairplot which shows relationship between variables captured by clustering
sns.pairplot(health_prop_LAD_ldn[['ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis','2016_mean','cluster']],
             hue='cluster',
             palette='Dark2',
            height=5);

In [ ]:
#groupby cluster to get the median value of each variable by cluster
health_prop_LAD_clusters_median=health_prop_LAD_ldn.groupby('cluster')[['ah4dent','ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis','2016_mean']].median().reset_index()

In [ ]:
#explore median values
health_prop_LAD_clusters_median

In [ ]:
# transform data to a long format to plot
health_prop_LAD_clusters_median_to_plot =pd.melt(health_prop_LAD_clusters_median,id_vars='cluster',
                                            value_vars=['ah4dent','ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis','2016_mean'])

In [ ]:
# check data
health_prop_LAD_clusters_median_to_plot.head()

In [ ]:
# Plot a faceted bar chart, where each row is a different cluster

sns.catplot(health_prop_LAD_clusters_median_to_plot,
            row='cluster',
            y='variable',
            x='value',
            kind='bar',
            aspect=4,
            hue='value',
            palette='summer');



In [ ]:
#changecluster naming
#create empty column
health_prop_LAD_ldn['Cluster_description']=""

In [ ]:
#update names of each cluster
health_prop_LAD_ldn.loc[health_prop_LAD_ldn['cluster']==0,'Cluster_description']='Medium Value Medium  Access'
health_prop_LAD_ldn.loc[health_prop_LAD_ldn['cluster']==1,'Cluster_description']='High Value Good Access'
health_prop_LAD_ldn.loc[health_prop_LAD_ldn['cluster']==2,'Cluster_description']='Medium High Value Medium Access'
health_prop_LAD_ldn.loc[health_prop_LAD_ldn['cluster']==3,'Cluster_description']='Low Value Lower Access'

# Spatial Visualisation
The second visualization provides two choropleth maps of equal size that illustrate the spatial variation in each of the four clusters in the first map, and in the second the weighted mean of health access for each LAD.

The intended audience for this being local decision makers to identify how their ward compares with others within London, a selection of stylistic and practical choices were made to ensure optimum accessibility and useability.

Initially the CRS of the data frame was converted to EPSG:3857 allowing its use with a basemap. The basemap selected was CartoDB from ‘contexttily’ under the option ‘Positron’. This map provided place names while remaining unobtrusive to the main visualization, and through adding a level of opacity to the choropleth maps, provides local context to the audience.  

Virdis color palette was used for the clusters as this was designed to be easily distinguishable by individuals with common forms of color blindness <a href="https://pos.sissa.it/guidelines.pdf">[8]</a> while still offering differentiation in color. Also, due to the nature of the data a not too strong sequential palette was preferable.

For the weighted mean access, magma was chosen as another palette that offers accessibility to people with common forms of colorblindness but differentiates it from the other choropleth map and offers a stronger sense of sequentiality <a href="https://matplotlib.org/stable/users/explain/colors/colormaps.html">[9]</a>.

Some final aesthetic choices that were made for useability were that on both maps’ axis were removed as these overcomplicated the plots, a North Arrow was added to aid in locality, the legend was moved to ensure none of the map area was covered, and finally the color bar was resized to offer enough detail while not overpowering the map.


In [ ]:
#Check CRS of datafame
health_prop_LAD_ldn.crs

In [ ]:
#convert crs to  Web Mercator projection (epsg=3857) to work with base maps
health_prop_LAD_ldn_WM = health_prop_LAD_ldn.to_crs(epsg=3857)

In [ ]:
#check new crs
health_prop_LAD_ldn_WM.crs

In [ ]:
# Data Visualisation

#create a figure with 3 subplots
f,ax = plt.subplots(1,2, figsize=(20,10), layout='constrained')
f.suptitle('Access Clusters and Mean Distance for London LADs ', fontsize=23)

#plot cluster description
health_prop_LAD_ldn_WM.plot(ax=ax[0], column ='Cluster_description', alpha=0.7, cmap='viridis', legend=True)
#add basemap
ctx.add_basemap(ax=ax[0], source=ctx.providers.CartoDB.Positron, crs=health_prop_LAD_ldn_WM.crs)
#plot LAD boundaries
health_prop_LAD_ldn_WM.boundary.plot(ax=ax[0], color='black', linewidth=.5)
# add title
ax[0].set_title('Individual Distance to Health Service Clusters', fontsize=20)
# add a North arrow
add_north_arrow(ax=ax[0], scale=.75, xlim_pos=0.1, ylim_pos=.85, color='#000', text_scaler=2, text_yT=-1.25)
# position legend
ax[0].get_legend().set_bbox_to_anchor((1.0, 0.99))
#plot LAD boundaries
health_prop_LAD_ldn_WM.boundary.plot(ax=ax[0], color='black', linewidth=1)


#plot health access score
health_prop_LAD_ldn_WM.plot(ax=ax[1], column ='ah4h', alpha=0.7, legend=True, cmap='magma',legend_kwds={'shrink': 0.7})
#add basemap
ctx.add_basemap(ax=ax[1], source=ctx.providers.CartoDB.Positron, crs=health_prop_LAD_ldn_WM.crs)
#plot LAD boundaries
health_prop_LAD_ldn_WM.boundary.plot(ax=ax[1], color='black', linewidth=.5)
# add title
ax[1].set_title('Weighted Mean Health Services Distance', fontsize=20)
# make axis invisible for subplot 1
ax[0].set_axis_off()
# make axis invisible for subplot 2
ax[1].set_axis_off()

plt.show();

#Results and Conclusions
From the exploration and visualization of this data we can draw some conclusions and observations. First is that there does appear to be a correlation between property value and access to health services although this is largely affected by scale.

At the smaller LSOA scale the correlation is far more variable and less conclusive, while at the aggregated LAD scale a negative correlation is much more pronounced.

Second is that at both scales’ dentists, leisure centers and pharmacies have a stronger correlation with house price than GPs or hospitals. This may be due to the private ownership of many of these facilities, where profitability is the focus.

Finally, we can observe that there is a spatial component to health access across London, with more central areas providing the best access, while access generally declines as you near the outskirts of the city.  


#Limitations
It is the hope that this project provides a useful exploration of how access to healthcare may affect house prices and highlights areas that may require more infrastructural support, however this report also comes with some limitations.

Chief among these is that the data only provides distance to service and not the capacity of the service itself <a href="https://www.nature.com/articles/s41597-019-0114-6">[1]</a>. Equally the list of services is not exhaustive, and the report could be improved in further inclusions.

Finally, the weighting of the overall distance is based on assumptions of the author and previous research, however exact weighting is difficult to justify <a href="https://www.nature.com/articles/s41597-019-0114-6">[1]</a> and therefore could be improved with further research.

Even with these limitations, however, access to healthcare is a right for all <a href="https://www.sciencedirect.com/science/article/pii/S0277953600004159">[10]</a> and not just for the wealthiest, and so in the future hopefully more work can be done on equality of healthcare across London.



## References
<p><a href="https://www.nature.com/articles/s41597-019-0114-6">[1]</a>Daras, K., Green, M.A., Davies, A., Barr, B. and Singleton, A., 2019. Open data on health-related neighbourhood features in Great Britain. Scientific data, 6(1), p.107.</p>

<p><a href="http://dx.doi.org/10.2139/ssrn.2785400">[2]</a> Kajuth, Florian and Schmidt, Tobias, Seasonality in House Prices (2011). Bundesbank Series 1 Discussion Paper No. 2011,08,</p>

<p><a href="https://www.sciencedirect.com/science/article/pii/S2772662223000048">[3]</a>Dash, C.S.K., Behera, A.K., Dehuri, S. and Ghosh, A., 2023. An outliers detection and elimination framework in classification task of data mining. Decision Analytics Journal, 6, p.100164.</p>

<p><a href="https://onlinelibrary.wiley.com/doi/10.1002/cpe.3745">[4]</a> Xiao, C., Ye, J., Esteves, R. M., and Rong, C. (2016) Using Spearman's correlation coefficients for exploratory data analysis on big dataset. Concurrency Computat.: Pract. Exper., 28: 3866–3878.</p>

<p><a href="https://ijcttjournal.org/archives/ijctt-v71i4p116">[5]</a>Srivastava, D., 2023. An introduction to data visualization tools and techniques in various domains. International Journal of Computer Trends and Technology, 71(4), pp.125-130.</p>

<p><a href="https://data.europa.eu/apps/data-visualisation-guide/handling-overlap-in-scatter-plots">[6]</a>European Data. 2023. Data-visualisation-guide handling-overlap-in-scatter-plots. Accessed July 2025</p>

<p><a href="http://166.62.7.99/assets/default/article/2020/10/22/article_1603378206.pdf">[7]</a>Cui, M., 2020. Introduction to the k-means clustering algorithm based on the elbow method. Accounting, Auditing and Finance, 1(1), pp.5-8.3</p>

<p><a href="https://pos.sissa.it/guidelines.pdf">[8]</a>Cherenkov Telescope Array Observatory (CTAO). (October, 2020). Best Practices for Colour Blind Friendly Publications & Descriptions.</p>

<p><a href="https://matplotlib.org/stable/users/explain/colors/colormaps.html">[9]</a>Matplotlib. Matplotlib development team. 2012-2025. Available at  https://matplotlib.org/stable/users/explain/colors/colormaps.html Accessed July 2025</p>

<p><a href="https://www.sciencedirect.com/science/article/pii/S0277953600004159">[10]</a>Goddard, M. and Smith, P., 2001. Equity of access to health care services:: Theory and evidence from the UK. Social science & medicine, 53(9), pp.1149-1162</p>


